# Object Syntactic Sugar

Some small modern features, which ease the implementation of your classes and objects.

## Construction improvement

#### Construction delegation

At last, one can now call a constructor from another constructor (for the same class).

In [ ]:
#include <iostream>

In [ ]:
class Int1
 {
  public :
    Int1() : Int1(0) { m_default = true ; }
    Int1( int i ) : m_val(i) { m_default = false ; }
    friend std::ostream & operator<<( std::ostream & os, const Int1 & i )
     { return (i.m_default?(os<<"(default)"<<i.m_val):(os<<"(explicit)"<<i.m_val)) ; }
  private :
    int m_val ;
    bool m_default ;
 } ;

In [ ]:
Int1 i1{}, i2{0} ;
std::cout << i1 << std::endl ;
std::cout << i2 << std::endl ;

#### Member variables initialisation

At last, one can give a default value to any member variable.

In [ ]:
struct Int2 {
    Int2() {}
    Int2( int x ) : m_x(x) {}
    int m_x = 0 ;
} ;

In [ ]:
Int2 i0 ;
Int2 i1 {1} ;
std::cout << i0.m_x << std::endl ;
std::cout << i1.m_x << std::endl ;

* This default initialisation can be modified within a constructor.
* Only the new initialization with curly braces is allowed here, not parenthesis.
* There is still no way to initialize a static non-constant member variable.

#### Constructors inheritance

At last, one can inherit the constructors from a base class, and do not need to manually replicate every and each base constructor.

In [ ]:
#include <iostream>

In [ ]:
struct A {
    A( int ) { std::cout << "A(int)" << std::endl ; }
    A( int, int ) { std::cout << "A(int,int)" << std::endl ; }
} ;

In [ ]:
struct B : public A
 {
  using A::A ; // on peut utiliser A::A( int) et A::A( int, int )
               // comme si ils étaient déclarés B::B( int) et B::B( int, int )
 } ;

In [ ]:
B b1{1} ;
B b2{1,2} ;

## Member functions improvement

#### Explicit conversion operators

At last, the keyword `explicit` is not any more only for constructors, but can also be used for conversion operators.

#### Forbid a function

By default, any class is given a copy member function, which sometimes does not make sense. In such a case, the ancient rule of thumb was to declare it private. The new syntax `= delete` better express the intention.

In [ ]:
// non-copyable class, C++03
class no_copies {
public:
    no_copies() {}
private:
    no_copies( no_copies const & ) ;
    no_copies & operator=( no_copies const & ) ;
} ;

In [ ]:
// non-copyable class, C++11
class no_copies_v2 {
public:
  no_copies_v2() {}
  no_copies_v2( const no_copies_v2 & ) = delete ;
  no_copies_v2 & operator=( const no_copies_v2 & ) = delete ;
} ;

Also, this can help prevent some unwanted implicit conversions.

In [ ]:
struct FooStruct {
    void foo_method(short) {}
    void foo_method(int) = delete ;
} ;

In [ ]:
FooStruct s ;
s.foo_method(42) ;        // Error, int overload declared deleted
s.foo_method((short)42) ; // OK

#### Enforce a function

On the contrary, for the methods which the compiler knows how to generate, one can enforce explicitly the code to be added, thanks to `= default`.

What for ?
* to make more visible a method which is implictly added.
* to make such a method protected or private, instead of public, yet keeping the usual implementation.
* to make a copy constructor handle a non-const input object.
* to make a virtual destructor, yet keeping the usual default implementation.
* to enforce the generation of the default constructor, while providing an additional user-made constructor.

In [ ]:
class Y {
  public:
    Y( Y & ) = default ; // Take a non-const reference
    Y & operator=( Y const & ) = default ; // Make it explicit
    virtual ~Y() = default ; // Add virtual
  private:
    Y() = default ; // Change access
} ;

Why is it better to use `= default`, rather than the simple equivalent implementation ? A user-made implementation may switch off some compiler optimizations and languages features :
* object copy with `memcpy` or `memmove`,
* object construction at compile time,
* use of the class in a customized union,
* use of the class with `std::atomic`.

#### Check virtual member functions overload

At last, when one intend to overload an inherited virtual function, one can add the keyword `override`, so that the compiler will check there is such a virtual function in a base class, with the exact same interface.

In [ ]:
class A
 {
  public :
    virtual void fct1() =0 ;
    virtual void fct2( int ) =0 ;
    virtual void fct3( bool ) =0 ;
 } ;

In [ ]:
class B : public A
 {
  public :
    void fct1 () override ;       // OK
    void ft2 ( int ) override ;   // erreur : A::ft2 n'existe pas
    void fct2 ( bool ) override ; // erreur : pas les bons types
 } ;

#### Forbid virtual member functions overload

Also, one can forbid any overload in further derived class, with the keyword `final`. With this same keyword, one can forbid a whole class to be derived.

In [ ]:
struct A {
    virtual void fct1() =0 ;
    virtual void fct2( int ) =0 ;
} ;

In [ ]:
struct B : public A {
    void fct3( int ) final ;
} ;

In [ ]:
struct C final : public B {
    void fct1 () override ;       // OK
    void fct3 ( bool ) override ; // erreur : B::fct3 est finale
} ;

In [ ]:
struct D : public C {} ;               // erreur : C est final         

## Exercise

In the code below, insert one `= delete`, one `= default` and one `override`.

In [ ]:
%%file tmp.objects-and-classes.cpp

#include <cstdlib> // pour std::rand()
#include <iostream>
#include <string>

class Particle
 {
  public  :
    Particle( double mass ) : m_mass(mass) {}
    double mass() { return m_mass ; }
    virtual std::string name() { return "Particle" ; }
    virtual ~Particle() {}
  private  :
    Particle( Particle const & ) ; // non copyable
    double m_mass ;
 } ;

class ChargedParticle : public Particle
 {
  public  :
    ChargedParticle( double mass, double charge )
     : Particle(mass), m_charge(charge) {}
    double charge() { return m_charge ; }
    virtual std::string name() { return "ChargedParticle" ; }
  private  :
    double m_charge ;
 } ;

void display( Particle & p  )
 {
  std::cout << p.name() << std::endl ;          
  std::cout << "  mass   = " << p.mass() << std::endl ;         
 }

int main()
 {
  for ( int i = 0 ; i < 5 ; ++i )
   {
    if ( std::rand() < (0.5 *  double(RAND_MAX)) )
     {
      Particle p(2) ;
      display(p) ;
     }
    else
     {
      ChargedParticle p(1,1) ;
      display(p) ;
      std::cout << "  charge = " << p.charge() << std::endl ;         
     }
   }
 }

In [ ]:
!rm -f tmp.objects-and-classes.exe && g++ -std=c++17 tmp.objects-and-classes.cpp -o tmp.objects-and-classes.exe

In [ ]:
!./tmp.objects-and-classes.exe

© *CNRS 2020*  
*Assembled and written by David Chamont, this work is made available according to the terms of the*  
[*Creative Commons License - Attribution - NonCommercial - ShareAlike 4.0 International*](http://creativecommons.org/licenses/by-nc-sa/4.0/)